In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import wandb
from architectures import BasicConvNet, ResNet18, MLP
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

# Exploring Tensorboard
Tensorboard is a local tool for visualizing images, metrics, histograms, and more. It is designed for tensorflow, but can be integrated with torch. Let's explore tensorboard usage with an example:

```
from torch.utils.tensorboard import SummaryWriter

# To start a run, call the following
writer = SummaryWriter(comment=f'Name_of_Run')

# When you want to log a value, use the writer. When adding a scalar, the format is as follows: 
# add_scalar(tag, scalar_value, global_step=None, walltime=None, new_style=False, double_precision=False)
writer.add_scalar('Training Loss', loss.item(), step)

# Finally, when you are done logging values, close the writer
writer.close()
```
There are many other functionalities and methods that you are free to explore, but will not be mentioned in this notebook.

## Your Task
We will be once again building classifiers for the CIFAR-10. There are various architectures set up for you to use in the architectures.py file. Using tensorboard, please search through 5 different hyperparameter configurations. Examples of choices include: learning rate, batch size, architecture, optimization algorithm, etc. Please submit the generated plots on your pdf and answer question A. 

In [2]:
epochs = 2
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                    download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
learning_rates = [1e-3, 1e-4]
batch_sizes = [128, 256]
log_freq = 2
eval_freq = 5

In [9]:
def run(learning_rate, batch_size):
    
    writer = SummaryWriter(comment=f'_LR_{learning_rate}_BS_{batch_size}')
    model = MLP().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)
    
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)
    
    
    
    
    step = 0
    
    for _ in range(int(epochs * 128/batch_size)):  # loop over the dataset multiple times
        dataiter = iter(testloader)
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            inputs = torch.flatten(inputs, start_dim=1)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            accuracy = torch.mean((torch.argmax(outputs, dim=1) == labels).float()).item() * 100


            if i % log_freq == 0:
                writer.add_scalar('Training Loss', loss.item(), step)
                writer.add_scalar('Training Accuracy', accuracy, step)
            if i % eval_freq == 0:
                eval_inputs, eval_labels = next(dataiter)
                eval_inputs, eval_labels = eval_inputs.to(device), eval_labels.to(device)
                eval_inputs = torch.flatten(eval_inputs, start_dim=1)
                model.eval()
                with torch.no_grad():
                    eval_outputs = model(eval_inputs)
                eval_loss = criterion(eval_outputs, eval_labels)
                model.train()
                eval_accuracy = torch.mean((torch.argmax(eval_outputs, dim=1) == eval_labels).float()).item() * 100
                writer.add_scalar('Validation Loss', eval_loss.item(), step)
                writer.add_scalar('Validation Accuracy', eval_accuracy, step)
            step += 1
            print(f'\rStep: {step}', end='')

   
    print(f'\nRun with (lr: {learning_rate}, batch_size: {batch_size}) has finished')
    writer.close()
            

In [10]:
for lr in learning_rates:
    for batch_size in batch_sizes:
        run(lr, batch_size)

Step: 782
Run with (lr: 0.001, batch_size: 128) has finished
